In [ ]:
# Cell 1: Setup and Parse Stock List
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from script import parse_stocks_file

ROOT = Path.cwd()
stocks_file = ROOT / 'stocks.txt'

# Parse stocks.txt to get individual tickers and baskets
individual_tickers, baskets = parse_stocks_file(stocks_file)

print(f"Individual tickers: {individual_tickers}")
print(f"\nBaskets: {baskets}")

In [ ]:
# Cell 2: Run Batch Analysis
from script import analyze_ticker, analyze_basket

def run_batch(individual_tickers, baskets, concurrency=6, daily_bars=60, weekly_bars=52):
    """Run analysis on all individual tickers and baskets"""
    results = []
    
    # Analyze individual tickers in parallel
    with ThreadPoolExecutor(max_workers=concurrency) as ex:
        futures = {ex.submit(analyze_ticker, t, daily_bars, weekly_bars): t for t in individual_tickers}
        for fut in as_completed(futures):
            results.append(fut.result())
    
    # Analyze baskets (market cap weighted aggregations)
    for basket_name, constituents in baskets.items():
        basket_result = analyze_basket(basket_name, constituents, daily_bars, weekly_bars)
        results.append(basket_result)
    
    return pd.DataFrame(results)

# Run the analysis
df = run_batch(individual_tickers, baskets)
df = df.round(2)

# Print Summary Stats
print(f"\nAnalysis complete: {len(df)} rows ({len(individual_tickers)} tickers + {len(baskets)} baskets)")
print("FULL HOLD + ADD tickers:", ', '.join(df[df['signal'] == 'FULL HOLD + ADD']['ticker']))

df

In [ ]:
# Cell 3: Print Buy Summary
def print_buy_summary(df, cash_available=118305):
    print("\n### Larsson Portfolio Buy Summary – Next Trading Day\n")
    print("**Rule-Based Only** — Conservative phased entry rules locked in:")
    print("- No immediate starters in Bullish zones (wait for better prices).")
    print("- Primary adds only on real weakness to predefined green supports.\n")
    
    print(f"**Cash Available**: ~${cash_available:,.0f} (~59% dry powder)\n")
    
    # Load target allocations from targets.csv
    targets_file = ROOT / 'targets.csv'
    if targets_file.exists():
        targets_df = pd.read_csv(targets_file)
        target_dict = dict(zip(targets_df['ticker'], targets_df['target_pct']))
        value_dict = dict(zip(targets_df['ticker'], targets_df['target_value']))
    else:
        target_dict = {}
        value_dict = {}
        print("⚠️  targets.csv not found - using N/A for target percentages\n")
    
    # Use exact match instead of contains
    eligible = df[df['signal'] == "FULL HOLD + ADD"]
    if eligible.empty:
        print("No FULL HOLD + ADD names — no buys recommended.")
        return
    
    def get_primary_zone(row):
        """Hybrid conservative primary add zone: Lower Value Area + Key Long-Term SMAs"""
        val = row['daily_val']
        poc = row['daily_poc']
        d100 = row['d100']
        d200 = row['d200']
        
        # Handle NaN/missing
        if pd.isna(val) or pd.isna(poc):
            return f"Near Key SMAs (D100 ${int(d100)} / D200 ${int(d200)})"
        
        lower_va = f"Lower Value Area (${int(val)}–${int(poc)})"
        sma_part = f"or Key Long-Term SMA (D100 ${int(d100)} / D200 ${int(d200)})"
        return f"{lower_va} {sma_part}"
    
    print("| Ticker | Target % | Current Price (Close) | Confluence | Buy Recommendation | Primary Add (40–50% of target) | Primary Add Zone (Conservative) | Approx Shares at Zone |")
    print("|--------|----------|-----------------------|------------|------------------------------|--------------------------------|---------------------------------|-----------------------|")
    
    for _, row in eligible.iterrows():
        ticker = row['ticker']
        price = row['current_price']
        confluence = row['confluence']
        rec = row['recommendation']
        
        # Get target % and value from config file
        target_pct = target_dict.get(ticker, 'N/A')
        target_val = value_dict.get(ticker, 4001)
        
        if target_pct != 'N/A':
            target_pct_str = f"{target_pct}%"
        else:
            target_pct_str = 'N/A'
        
        # Calculate primary add amounts (40-50% of target)
        primary_low = target_val * 0.4
        primary_high = target_val * 0.5
        
        # Get primary zone
        zone = get_primary_zone(row)
        
        # Share estimate using approximate zone midpoint (or current price fallback)
        zone_mid_est = price * 0.9  # rough 10% dip estimate for conservatism
        shares_low = int(primary_low / zone_mid_est)
        shares_high = int(primary_high / zone_mid_est)
        shares_str = f"{shares_low}–{shares_high} shares"
        
        print(f"| **{ticker}** | {target_pct_str} | ${price:.2f} | **{confluence}** | {rec} | ~${primary_low:,.0f}–${primary_high:,.0f} | {zone} | {shares_str} |")
    
    print("\n**No Buy Action**")
    print("- All other names: Not FULL HOLD + ADD or confluence insufficient.\n")
    print("**Execution Plan**")
    print("- No limits to place / Wait for weakness to primary zones / etc.")

print_buy_summary(df)


In [ ]:
# Cell 4: Export Results
import shutil
from datetime import datetime

# Add timestamp to avoid overwriting
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_file = ROOT / f'batch_results_{timestamp}.csv'

# Save to workspace
df.to_csv(output_file, index=False)
print(f"✅ Results exported to: {output_file}")

# Copy to Downloads folder
downloads_path = Path.home() / 'Downloads' / f'batch_results_{timestamp}.csv'
try:
    shutil.copy(output_file, downloads_path)
    print(f"✅ File also copied to: {downloads_path}")
except PermissionError:
    print(f"⚠️  Could not copy to Downloads - file may be open in another program")
    print(f"   Close the file and try again, or use: {output_file}")

In [ ]:
# Cell 5: Cleanup Old Results
from pathlib import Path

def cleanup_old_results(keep_latest=1):
    """Keep only the most recent batch result file"""
    batch_files = sorted(
        ROOT.glob('batch_results_*.csv'),
        key=lambda p: p.stat().st_mtime,
        reverse=True
    )
    
    for old_file in batch_files[keep_latest:]:
        print(f"🗑️  Deleting: {old_file.name}")
        old_file.unlink()
    
    print(f"✅ Kept {min(len(batch_files), keep_latest)} most recent file(s)")

# Auto-cleanup after each run:
cleanup_old_results(keep_latest=1)